# Basic Usage Example

This notebook demonstrates the basic usage of `claude-message-visualizer`.

## Setup

**Recommended:** Create a `.env` file in your project root:
```
ANTHROPIC_API_KEY=your-api-key-here
```

This file is already in `.gitignore` so your key won't be committed to git.

The code below uses `python-dotenv` to load the key securely from your `.env` file.

In [14]:
# Install if needed
# !pip install claude-message-visualizer anthropic python-dotenv

In [15]:
from dotenv import load_dotenv
from anthropic import Anthropic
from claude_message_visualizer import visualize_message

load_dotenv()
client = Anthropic()

## Example 1: Simple Text Response

In [16]:
response = client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=100,
    messages=[{
        "role": "user",
        "content": "Hello, Claude!"
    }]
)

visualize_message(response)

## Example 2: Longer Response with Auto-collapse

In [17]:
response = client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=2000,
    messages=[{
        "role": "user",
        "content": "Write a detailed explanation of how neural networks work, including backpropagation."
    }]
)

# Long text blocks (>1000 chars) are automatically collapsible
visualize_message(response)

## Example 3: Extended Thinking

This example shows how the visualizer displays Claude's internal reasoning process. The thinking block is collapsible and shows the character count.

In [18]:
response = client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=8000,
    thinking={
        "type": "enabled",
        "budget_tokens": 5000
    },
    messages=[{
        "role": "user",
        "content": "Design a spam detection system using machine learning. What are the key challenges?"
    }]
)

visualize_message(response)

## Example 4: Tool Use Complete Workflow

This example shows the complete tool use cycle:
1. Claude requests to use a tool (`tool_use` block)
2. Your application executes the tool and provides results (`tool_result` block in message)
3. Claude processes the result and responds (`text` block)

This demonstrates how tools enable Claude to interact with external systems.

In [22]:
from anthropic.types import ToolParam

# Define weather tool
tools: list[ToolParam] = [{
    "name": "get_weather",
    "description": "Get current weather for a location",
    "input_schema": {
        "type": "object",
        "properties": {
            "location": {"type": "string", "description": "City name"}
        },
        "required": ["location"]
    }
}]

# Step 1: User asks a question, Claude decides to use tool
user_message = "What's the weather in San Francisco?"

print("=" * 60)
print("STEP 1: User asks question")
print("=" * 60)
print(f"User: {user_message}\n")

response1 = client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1000,
    tools=tools,
    messages=[{"role": "user", "content": user_message}]
)

print("Claude's response (requests tool use):")
visualize_message(response1)

# Step 2: App executes tool and provides result, Claude synthesizes answer
tool_use_block = next(block for block in response1.content if block.type == "tool_use")
tool_result_content = "The weather in San Francisco is currently 62°F (17°C), partly cloudy with light winds."

print("\n" + "=" * 60)
print("STEP 2: Application provides tool result")
print("=" * 60)
print(f"Tool Result: {tool_result_content}\n")

response2 = client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1000,
    tools=tools,
    messages=[
        {"role": "user", "content": user_message},
        {"role": "assistant", "content": response1.content},
        {
            "role": "user",
            "content": [
                {
                    "type": "tool_result",
                    "tool_use_id": tool_use_block.id,
                    "content": tool_result_content
                }
            ]
        }
    ]
)

print("Claude's final response:")
visualize_message(response2)

STEP 1: User asks question
User: What's the weather in San Francisco?

Claude's response (requests tool use):



STEP 2: Application provides tool result
Tool Result: The weather in San Francisco is currently 62°F (17°C), partly cloudy with light winds.

Claude's final response:


## Example 5: Web Search with Citations

This example shows the visualizer's citation feature. When Claude uses web search and cites sources, the visualizer displays:
- Web search results as collapsible cards
- Citations with red-highlighted quoted text
- Automatic citation numbering linked to search results

In [20]:
response = client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=2000,
    tools=[{"type": "web_search_20250305", "name": "web_search", "max_uses": 5}],
    messages=[{
        "role": "user",
        "content": "What are the latest recommendations for vitamin D intake? Please cite your sources."
    }]
)

visualize_message(response)

## Next Steps

Now that you've seen the visualizer's features, try:
- Modify the examples above with your own prompts
- Experiment with different tool definitions
- Combine features (e.g., extended thinking + web search)

For more information:
- [Full Documentation](../README.md)
- [Report Issues](https://github.com/frankiewarren/claude-message-visualizer/issues)
- [Claude API Docs](https://docs.anthropic.com/)